In [41]:
import numpy as np
import pickle
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import sys
from scipy.interpolate import griddata
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.colorbar
from matplotlib import rc
from scipy.optimize import fsolve
from scipy.integrate import simpson
from numpy import trapz
from numpy.polynomial.polynomial import polyfit
from scipy import integrate

In [42]:
def fermi_distribution(en,mu,beta):
    fd = 0
    if beta>500:
       if en<=mu:
          fd = 1
       if en>mu:
          fd  = 0

    else:
       fd = 1.0/(np.exp(beta*(en-mu))+1.0)
    
    return fd

In [43]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx,array[idx]

In [44]:
def radius_theta_cons(kx_mesh,ky_mesh):
   
  
   r = np.zeros(kx_mesh.shape)
   theta = np.zeros(ky_mesh.shape)
   for i in range(len(r[:,0])):
       for j in range(len(r[0,:])):
           r[i][j] = np.abs(np.sqrt((kx_mesh[i][j])**2+(ky_mesh[i][j])**2))
           if(r[i][j]>0):
              if(kx_mesh[i][j] == 0):
                 if(ky_mesh[i][j]>0):
                    theta[i][j] = np.arctan(np.inf)
                 if(ky_mesh[i][j]<0):
                    theta[i][j] = np.arctan(-1*np.inf)
              if(kx_mesh[i][j]>0):
                    theta[i][j] = np.arctan(ky_mesh[i][j]/kx_mesh[i][j])
              if(kx_mesh[i][j]<0):
                    theta[i][j] = np.pi+np.arctan(ky_mesh[i][j]/kx_mesh[i][j])

   return r,theta

In [45]:
def number_density_calc(chemical_potential,beta):

   initial_x = 0
   initial_y = 0
   final_x = np.pi
   final_y = np.pi

   dx = np.pi/50
   dy = np.pi/50

   x_grid_cord = np.arange(-np.pi,np.pi,dx)
   y_grid_cord = np.arange(-np.pi,np.pi,dy)

   Ek = 0 #np.zeros((len(x_grid_cord),len(y_grid_cord)))
   nk_noninteracting = 0 #np.zeros((len(x_grid_cord),len(y_grid_cord)))

   t = 1
   bz_area = 0

   for i in range(len(x_grid_cord)):
       for j in range(len(y_grid_cord)):
           Ek = -2*t*(np.cos(x_grid_cord[i])+np.cos(y_grid_cord[j]))
           nk_noninteracting = nk_noninteracting+2*fermi_distribution(Ek,chemical_potential,beta)
           bz_area = bz_area+1

   Nk = nk_noninteracting/bz_area
   return Nk

In [46]:
def brillouin_zone_labels(Text_dir,N):
   filename_k = '%s/Brillouin_zone_co-oridinates_N_%s.pkl' %(Text_dir,N)
   with open(filename_k, 'rb') as infile:
        K = pickle.load(infile)
  
   kx = K[:,0]
   ky = K[:,1]
   K_grid = np.zeros((3,len(kx)))
   
   for j in range(len(kx)):
       K_grid[0][j] = j
       K_grid[1][j] = kx[j]
       K_grid[2][j] = ky[j]
   print(K_grid,"BZ")
   return K_grid

In [47]:
def k_point_grid_upper_half_bz(N):
    K_label = []
    Kx = []
    Ky = []
    x_span = int((int(N))/2)+1
    k_pair = 0
    for i in range(x_span):
        for j in range(x_span):
            if(j>=i):
              K_label.append(k_pair)  
              Kx.append(i)
              Ky.append(j)
              k_pair = k_pair+1

              
    BZ = np.stack((K_label,Kx,Ky),axis = 1)
    
    return BZ,k_pair

In [48]:
def k_point_grid_full_bz(N):
    K_label = []
    Kx = []
    Ky = []
    x_span = int((int(N))/2)+1
    k_pair = 0
    for i in range(x_span):
        for j in range(x_span):
              K_label.append(k_pair)
              Kx.append(i)
              Ky.append(j)
              k_pair = k_pair+1
              
    BZ = np.stack((K_label,Kx,Ky),axis = 1)

    return BZ,k_pair

In [49]:
def k_point_pi_pi_locator(klabel,kxgrid,kygrid):
 
    k_lab_pi_pi = []
    kx_pi_pi = []
    ky_pi_pi = []
    for i in range(len(kxgrid)):
        if kxgrid[i] == kygrid[i]:
           k_lab_pi_pi.append(klabel[i])
           kx_pi_pi.append(kxgrid[i])
           ky_pi_pi.append(kygrid[i]) 
            
    return k_lab_pi_pi,kx_pi_pi,ky_pi_pi

In [50]:
def number_density_func(mu_val,num_den,beta):

   y = 0

   initial_x = -np.pi
   initial_y= -np.pi
   final_x = np.pi
   final_y = np.pi

   dx = 2*np.pi/50
   dy = 2*np.pi/50

   x_grid_cord = np.arange(-np.pi,np.pi,dx)
   y_grid_cord = np.arange(-np.pi,np.pi,dy)

   Ek = 0 #np.zeros((len(x_grid_cord),len(y_grid_cord)))
   nk_noninteracting = 0 #np.zeros((len(x_grid_cord),len(y_grid_cord)))

   t = 1
   bz_area = 0

   for i in range(len(x_grid_cord)):
       for j in range(len(y_grid_cord)):
           Ek = -2*t*(np.cos(x_grid_cord[i])+np.cos(y_grid_cord[j]))
           nk_noninteracting = nk_noninteracting+2*fermi_distribution(Ek,mu_val,beta)
           bz_area = bz_area+1
   
   Nk = nk_noninteracting/bz_area
   y = Nk-num_den

   return y

In [51]:
def adj_mat_calc(N):

    r_grid, r_rel_grid_size = k_point_grid_upper_half_bz(int(N))
    r_rel_x = r_grid[:,0]
    r_rel_y = r_grid[:,1]

    r_2 = np.add(np.power(r_rel_x,2),np.power(r_rel_y,2))
    r_2_unq = np.sort(np.unique(r_2))

    rx_cord = []
    ry_cord = []
    r2 = []
    rx = np.arange(int(int(N)/2),-1*int(int(N)/2),-1)
    ry = np.arange(int(int(N)/2),-1*int(int(N)/2),-1)
    for i in range(len(rx)):
        for j in range(len(ry)):
            rx_cord.append(rx[i])
            ry_cord.append(ry[j])
            r2.append(rx[i]*rx[i]+ry[j]*ry[j])

    #plt.figure()
    #plt.scatter(rx_cord,ry_cord,c=r2)

    adj_mat = np.zeros(len(r_2_unq))
    for i in range(len(r_2_unq)):
        count = 0
        for i2 in range(len(r2)):
            if round(r2[i2],2) == round(r_2_unq[i],2):
               count = count+1
        adj_mat[i] = count
    
    return np.stack((r_2_unq,np.asarray(adj_mat)),axis = 1)  

In [52]:
def LDOS_plot_v2(Graph_dir_main,freq_no,sigma,N,u,Mu,trot_slice,dtau,omega_max,omega_min):

   Adj_list = adj_mat_calc(N)
   beta = float(dtau)*float(trot_slice)
   T = 1/beta
   
   bz,k_pairs = k_point_grid_upper_half_bz(N)
   k_label = bz[:,0]
   Kx_grid = bz[:,1]
   Ky_grid = bz[:,2]

   Nden = np.zeros(len(Mu))
   LDOS_gauss = np.zeros((len(Mu),freq_no))

   for j in range(len(Mu)): 
                
       Text_dir_eqm = "/Users/roy.369/Documents/Luttinger_theorem/FHM_Legacy_data/FHM_Roy_data/Text_files/Text_files_N_%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Thermodynamic_measurements"%(N,N,u,dtau,Mu[j],dtau,trot_slice)
       filename_eqm_avg = '%s/Thermodynamic_measurements_normal_averaged_dictionary_N_%s_U_%s_mu_%s_dtau_%s_L_%s.pkl' %(Text_dir_eqm,N,u,Mu[j],dtau,trot_slice)
       with open(filename_eqm_avg, 'rb') as infile:
            sys_measure_avg = pickle.load(infile)

       filename_eqm_std = '%s/Thermodynamic_measurements_normal_standard_deviation_dictionary_N_%s_U_%s_mu_%s_dtau_%s_L_%s.pkl' %(Text_dir_eqm,N,u,Mu[j],dtau,trot_slice)
       with open(filename_eqm_std, 'rb') as infile:
            sys_measure_std = pickle.load(infile)

       Density = sys_measure_avg['Density averaged']
       Density_dev = sys_measure_std['Density standard deviation']
       Avg_sign = sys_measure_avg['Total sign averaged']   
       Nden[j] = Density
       
       Omega_gauss = np.zeros((len(k_label),freq_no))
       A_k_gauss = np.zeros((len(k_label),freq_no))

       #======================Read the maxent data files one by one==========================================
       for kk in range(len(k_label)):
           
           Text_dir_gf_k_avg_ac_main = "/Users/roy.369/Documents/Luttinger_theorem/FHM_Legacy_data/FHM_Roy_data/Text_files/Text_files_N_%s_analytic_continuation_spectral_functions/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Spectral_functions_gaussian_sigma_%s_nfreq_%s"%(N,N,u,dtau,Mu[j],dtau,trot_slice,sigma,str(freq_no))
                
           k_point = str(int(k_label[kk]))
           filename_a_k_avg_ac_gauss = '%s/k_point_%s/Spectral_function_momentum_space_avg_N_%s_U_%s_mu_%s_dtau_%s_L_%s_k_label_%s_gaussian_sigma_%s_nfreq_%s_omega_max_%s_omega_min_%s.dat' %(Text_dir_gf_k_avg_ac_main,k_point,N,u,mu,dtau,trot_slice,k_point,sigma,str(freq_no),omega_max,omega_min)
           omega_gauss,a_k_gauss = np.loadtxt(filename_a_k_avg_ac_gauss,unpack = 'True',usecols = [0,1])
           Omega_gauss[kk,:] = np.copy(omega_gauss)
           A_k_gauss[kk,:] = np.copy(a_k_gauss)
           
       for w in range(len(Omega_gauss[0,:])):
            LDOS_gaus[j,w] = np.dot(A_k_gauss[:,w],Adj_list[:,1])

   Graph_dir_LDOS = "%s/Maxent_gaussian_default_sigma_%s_LDOS_N_%s_U_%s_dtau_%s"%(Graph_dir_main,sigma,N,u,dtau)
   if not os.path.exists(Graph_dir_LDOS):
       os.makedirs(Graph_dir_LDOS)
        
   plt.figure(figsize = (25,20))
   plt.xticks(fontsize = 80)
   plt.yticks(fontsize = 80)
   for j in range(len(Mu)):
       plt.plot(Omega[0,:],LDOS_gaus[j,:],marker = "o",markersize = 20, label="n=%s"%str(round(Nden[j],3)))
   plt.grid(True,which='both')
   plt.legend(loc = 'best',fontsize = 50)
   plt.savefig("%s/Maxent_LDOS_gaussian_sigma_%s_N_%s_U_%s_dtau_%s_L_%s.png"%(Graph_dir_LDOS,N,u,dtau,trot_slice))
   

In [101]:
def luttinger_surface_spectral_functions_full_bz_plot_v2(Text_dir_gf_k_avg_ac_main,Graph_dir_main,n_den,freq_no,sigma,N,u,mu,trot_slice,dtau,omega_max,omega_min,x_grid_size,y_grid_size):

   eta = 0.000001*1j
    
   beta = float(dtau)*float(trot_slice)
   T = 1/beta
   
   mu_ni_actual = fsolve(number_density_func,[0.0],args = (n_den,beta))
   nd_ni_actual = number_density_calc(mu_ni_actual[0],beta)

   print(mu_ni_actual,nd_ni_actual,n_den,"Non_interacting, interacting density")
   bz,k_pairs = k_point_grid_upper_half_bz(N)
   k_label_0 = bz[:,0]
   Kx_grid_0 = bz[:,1]
   Ky_grid_0 = bz[:,2]

   k_label_1 = []
   Kx_grid_1 = []
   Ky_grid_1 = []


   for ii in range(len(k_label_0)):

       if(Kx_grid_0[ii] != Ky_grid_0[ii]):
          kx = Ky_grid_0[ii]
          ky = Kx_grid_0[ii]
          Kx_grid_1.append(kx)
          Ky_grid_1.append(ky)
          k_label_1.append(k_label_0[ii])


   k_label = np.append(k_label_0,np.array(k_label_1))
   Kx_grid_2 = np.append(Kx_grid_0,np.array(Kx_grid_1))
   Ky_grid_2 = np.append(Ky_grid_0,np.array(Ky_grid_1))

   Kx_grid = (2*np.pi/int(N))*Kx_grid_2
   Ky_grid = (2*np.pi/int(N))*Ky_grid_2

   K_grid = np.stack((Kx_grid,Ky_grid),axis = 1)
   
   initial_x = np.nanmin(Kx_grid)
   initial_y= np.nanmin(Ky_grid)
   final_x = np.nanmax(Kx_grid)
   final_y = np.nanmax(Ky_grid)

   x_grid_cord = np.linspace(initial_x,final_x,num = x_grid_size)
   y_grid_cord = np.linspace(initial_y,final_y,num = y_grid_size)
   Kx_mesh,Ky_mesh = np.meshgrid(x_grid_cord,y_grid_cord,indexing = 'xy')
   R,Phi = radius_theta_cons(Kx_mesh,Ky_mesh)
   
   dx = Kx_mesh[0,:]
   dy = Ky_mesh[:,0]

   x_grid_cord_ni = np.linspace(0,np.pi,50)
   y_grid_cord_ni = np.linspace(0,np.pi,50)

   T = 1/beta
   t = 1
   Kx_mesh_ni,Ky_mesh_ni = np.meshgrid(x_grid_cord_ni,y_grid_cord_ni,indexing = 'xy')
   A_k_NI = np.zeros(Kx_mesh_ni.shape)
   Ek_NI =  np.zeros(Kx_mesh_ni.shape)
   for i in range(len(Ky_mesh_ni[:,0])):
       for j in range(len(Kx_mesh_ni[0,:])):
           Ek_NI[j][i] = -2*t*(np.cos(Kx_mesh_ni[j][i])+np.cos(Ky_mesh_ni[j][i]))
           Epsilon_kk = Ek_NI[j][i]-mu_ni_actual[0]
           A_k_NI[j][i] = 0.5/(np.cosh((beta*Epsilon_kk)/2))
      
   n_k = np.zeros(len(k_label))
   G_k_b2 = np.zeros(len(k_label))
   Omega_gaus = np.zeros((len(k_label),freq_no))
   A_k_gaus = np.zeros((len(k_label),freq_no))
   Luttinger_surface_gaus = np.zeros((len(k_label)))
   FS_gaus = np.zeros((len(k_label)))


   #======================Read the maxent data files one by one==========================================
   for kk in range(len(k_label)):

       k_point = str(int(k_label[kk]))
       filename_gf_k_avg = '%s/k_point_%s/Retarded_green_function_momentum_space_avg_N_%s_U_%s_mu_%s_dtau_%s_L_%s_k_label_%s_ac.dat' %(Text_dir_gf_k_avg_ac_main,k_point,N,u,mu,dtau,trot_slice,k_point)
       tau,g_k,g_k_std = np.loadtxt(filename_gf_k_avg,unpack = 'True',usecols = [0,1,2])
       n_k[kk] = 1+g_k[0]
       lp = int(int(trot_slice)/2)
       G_k_b2[kk] = -beta*g_k[lp]/np.pi
       
       filename_a_k_avg_ac_gaus = '%s/k_point_%s/Spectral_function_momentum_space_avg_N_%s_U_%s_mu_%s_dtau_%s_L_%s_k_label_%s_gaussian_sigma_%s_nfreq_%s_omega_max_%s_omega_min_%s.dat' %(Text_dir_gf_k_avg_ac_main,k_point,N,u,mu,dtau,trot_slice,k_point,sigma,str(freq_no),omega_max,omega_min)
       omega_gaus,a_k_gaus = np.loadtxt(filename_a_k_avg_ac_gaus,unpack = 'True',usecols = [0,1])

       Omega_gaus[kk,:] = np.copy(omega_gaus)
       A_k_gaus[kk,:] = np.copy(a_k_gaus)
       zero_ind_gaus,zero_val_gaus = find_nearest(omega_gaus,0.00000001)
       Luttinger_surface_gaus[kk] = np.real(-1*integrate.trapz(np.divide(A_k_gaus[kk,:],np.subtract(eta*np.ones(freq_no),Omega_gaus[kk,:])),Omega_gaus[kk,:]))
       FS_gaus[kk] = (a_k_gaus[zero_ind_gaus]+a_k_gaus[zero_ind_gaus-1])/2

    
   #=============================Let's perform cubic interpolation of the data============================ 

   grid_g_k_c = griddata(K_grid,G_k_b2,(Kx_mesh,Ky_mesh),method = 'cubic')
   grid_FS_k_c = griddata(K_grid,FS_gaus,(Kx_mesh,Ky_mesh),method = 'cubic')
   grid_LS_k_c = griddata(K_grid,Luttinger_surface_gaus,(Kx_mesh,Ky_mesh),method = 'cubic')
       
   mdf_actual,ind_min = find_nearest(n_k,0.5)
   
   dx_ni = np.pi/50
   dy_ni = np.pi/50


   #==========================Radial derivative to find maximum of contours===============================

   del_x_grid_A_k_NI = np.gradient(A_k_NI,dx_ni,axis = 1)
   del_y_grid_A_k_NI = np.gradient(A_k_NI,dy_ni,axis = 0)
   del_x_grid_g_k_c = np.gradient(grid_g_k_c,dx,axis = 1)
   del_y_grid_g_k_c = np.gradient(grid_g_k_c,dy,axis = 0)
   del_x_grid_FS_k_c = np.gradient(grid_FS_k_c,dx,axis = 1)
   del_y_grid_FS_k_c = np.gradient(grid_FS_k_c,dy,axis = 0)

   radial_del_NI = np.zeros(A_k_NI.shape)
   radial_g_k_del = np.zeros(grid_g_k_c.shape)
   radial_FS_del = np.zeros(grid_FS_k_c.shape)
   
   for i in range(len(Ky_mesh[:,0])):
       for j in range(len(Kx_mesh[0,:])):

           radial_FS_del[j][i] = (np.cos(Phi[j][i]))*(del_x_grid_FS_k_c[j][i])+(np.sin(Phi[j][i]))*(del_y_grid_FS_k_c[j][i])
           radial_g_k_del[j][i] = (np.cos(Phi[j][i]))*(del_x_grid_g_k_c[j][i])+(np.sin(Phi[j][i]))*(del_y_grid_g_k_c[j][i])
   
   R_ni,Phi_ni = radius_theta_cons(Kx_mesh_ni,Ky_mesh_ni)
   for i in range(len(Ky_mesh_ni[:,0])):
       for j in range(len(Kx_mesh_ni[0,:])):
           radial_del_NI[j][i] = (np.cos(Phi_ni[j][i]))*(del_x_grid_A_k_NI[j][i])+(np.sin(Phi_ni[j][i]))*(del_y_grid_A_k_NI[j][i])

   
   #===================Calculating Fermi surface and Luttinger surface contours for gaussian spectral functions====================================================================
   
   x_cord_loc = np.arange(0,int(N)/2+1,1)
   y_cord_loc = np.arange(0,int(N)/2+1,1)
    
   max_a_k = np.nanmax(FS_gaus)

   cm = matplotlib.colormaps.get_cmap('plasma')

   Graph_dir_gaus_fs_scat = "%s/Gaussian_sigma_%s_fermi_surface_spectral_functions_scattered"%(Graph_dir_main,sigma)
   if not os.path.exists(Graph_dir_gaus_fs_scat):
      os.makedirs(Graph_dir_gaus_fs_scat)
   
   Graph_dir_gaus_ls_scat = "%s/Gaussian_sigma_%s_luttinger_surface_spectral_functions_scattered"%(Graph_dir_main,sigma)
   if not os.path.exists(Graph_dir_gaus_ls_scat):
      os.makedirs(Graph_dir_gaus_ls_scat)

   Graph_dir_gaus_fs_cont = "%s/Gaussian_sigma_%s_fermi_surface_spectral_functions_cubic_interpolation_contour"%(Graph_dir_main,sigma)
   if not os.path.exists(Graph_dir_gaus_fs_cont):
      os.makedirs(Graph_dir_gaus_fs_cont)
       
   Graph_dir_gaus_ls_cont = "%s/Gaussian_sigma_%s_luttinger_surface_spectral_functions_cubic_interpolation_contour"%(Graph_dir_main,sigma)
   if not os.path.exists(Graph_dir_gaus_ls_cont):
      os.makedirs(Graph_dir_gaus_ls_cont)

   Graph_dir_gaus_ls_cont_data_fit = "%s/Gaussian_sigma_%s_luttinger_surface_spectral_functions_cubic_interpolation_contour/Polynomial_contour_fit"%(Graph_dir_main,sigma)
   if not os.path.exists(Graph_dir_gaus_ls_cont_data_fit):
      os.makedirs(Graph_dir_gaus_ls_cont_data_fit)


   #idx = np.argmax(grid_FS_k_c, axis=1)
# use idx to get the x-axis values from T that correspond to max XA
   #x = np.take_along_axis(Kx_mesh, np.expand_dims(idx, axis=-1), axis=-1).squeeze(axis=-1)
# use idx to get the max y-axis values from XA
   #y = np.take_along_axis(Ky_mesh, np.expand_dims(idx, axis=-1), axis=-1).squeeze(axis=-1)

   plt.figure(figsize = (20,20))
   plt.title(r"$A (k,\omega=0), N = %sx%s, U = %s, \mu = %s, n=%s, T = %s$"%(N,N,u,mu,str(round(n_den,4)),str(round(T,3))),fontsize = 40)
   ax = plt.gca()
   plt.xlabel('Kx',fontsize = 40)
   plt.ylabel('Ky',fontsize = 40)
   scat = plt.scatter(Kx_grid,Ky_grid,c=FS_gaus,marker = "s",s = 40000,cmap=cm)
   plt.xticks(fontsize = 40)
   plt.yticks(fontsize = 40)
   plt.grid('True',linewidth=3)
   divider = make_axes_locatable(ax)
   cax = divider.append_axes("right", size="5%", pad=0.05)
   cbar = plt.colorbar(scat, cax= cax)
   cbar.ax.tick_params(labelsize = 30)
   plt.savefig("%s/Gaussian_sigma_%s_spectral_function_N_%s_U_%s_mu_%s_dtau_%s_L_%s_omega_max_%s_omega_min_%s_scatter.png"%(Graph_dir_gaus_fs_scat,sigma,N,u,mu,dtau,trot_slice,omega_max,omega_min))
   plt.close('all') 

    
   plt.figure(figsize = (20,20))
   plt.title(r"$Re G (k,\omega=0), N = %sx%s, U = %s, \mu = %s, n=%s, T = %s$"%(N,N,u,mu,str(round(n_den,4)),str(round(T,3))),fontsize = 40)
   ax = plt.gca()
   plt.xlabel('Kx',fontsize = 40)
   plt.ylabel('Ky',fontsize = 40)
   scat = plt.scatter(Kx_grid,Ky_grid,c=Luttinger_surface_gaus,marker = "s",s = 40000,cmap=cm)
   plt.xticks(fontsize = 40)
   plt.yticks(fontsize = 40)
   plt.grid('True',linewidth=3)
   divider = make_axes_locatable(ax)
   cax = divider.append_axes("right", size="5%", pad=0.05)
   cbar = plt.colorbar(scat, cax= cax)
   cbar.ax.tick_params(labelsize = 30)
   plt.savefig("%s/Gaussian_sigma_%s_real_part_green_function_N_%s_U_%s_mu_%s_dtau_%s_L_%s_omega_max_%s_omega_min_%s_scatter.png"%(Graph_dir_gaus_ls_scat,sigma,N,u,mu,dtau,trot_slice,omega_max,omega_min))
   plt.close('all') 

    
   plt.figure(figsize = (28,25))
   #plt.title(r"$Re G(k,\omega=0), N = %sx%s, U = %s, \mu=%s, n =%s, T=%s$" %(N,N,u,mu,str(round(n_den,4)),str(round(T,3))),fontsize = 40)
   ax = plt.gca()
   #plt.xlabel(r"$K_x$",fontsize = 80)
   #plt.ylabel(r"$K_y$",fontsize = 80)
   ntext="n=%s"
   plt.xticks(np.round(2*np.pi/int(N),2)*x_cord_loc,fontsize = 80)
   plt.yticks(np.round(2*np.pi/int(N),2)*y_cord_loc,fontsize = 80)
   im = plt.contourf(Kx_mesh,Ky_mesh,grid_LS_k_c,levels=50,cmap = cm)
   #plt.text(2.3, 2.9, r"$n=%s$"%str(round(n_den,3)), c='black', fontsize=80)
   plt.grid('True',linewidth=2)
   #CS = ax.contour(Kx_mesh, Ky_mesh, grid_LS_k_c, [0],colors=['black'],linewidths = [5])
   #plt.plot(x, y, c = 'green', linestyle = 'dashdot',linewidth = 5)
   #CS_1 = ax.contour(Kx_mesh, Ky_mesh, radial_FS_del, [0.0], colors = ['green'], linestyles = ['dashdot'],linewidths = [5])
   #CS_2 = ax.contour(Kx_mesh, Ky_mesh, radial_g_k_del, [0.0], colors = ['orange'], linestyles = ['dashdot'],linewidths = [5])
   #CS_3 = ax.contour(Kx_mesh_ni, Ky_mesh_ni, Ek_NI, [mu_ni_actual], colors = ['white'], linestyles = ['dashdot'],linewidths = [5])
   divider = make_axes_locatable(ax)
   cax = divider.append_axes("right", size="5%", pad=0.05)
   cbar = plt.colorbar(im, cax= cax)
   cbar.ax.tick_params(labelsize = 60)
   plt.tight_layout()
   plt.savefig("%s/Gaussian_sigma_%s_spectral_functions_luttinger_surface_N_%s_U_%s_mu_%s_dtau_%s_L_%s_omega_max_%s_omega_min_%s.png"%(Graph_dir_gaus_ls_cont,sigma,N,u,mu,dtau,trot_slice,omega_max,omega_min))
   plt.close('all')


   print(x_cord_loc)
   plt.figure(figsize = (28,25))
   #plt.title(r"$A(k,\omega=0), N = %sx%s, U = %s, \mu=%s, n =%s, T=%s$" %(N,N,u,mu,str(round(n_den,4)),str(round(T,3))),fontsize = 40)
   ax = plt.gca()
   #plt.xlabel(r"$K_x$",fontsize = 80)
   #plt.ylabel(r"$K_y$",fontsize = 80)
   ntext="n=%s"
   #plt.xticks(np.round(2*np.pi/int(N),2)*x_cord_loc,fontsize = 80)
   #plt.yticks(np.round(2*np.pi/int(N),2)*y_cord_loc,fontsize = 80)
   im = plt.contourf(Kx_mesh,Ky_mesh,grid_FS_k_c,levels=50,cmap = cm)
   #plt.text(2.3, 2.9, r"$n=%s$"%str(round(n_den,3)), c='black', fontsize=80)
   plt.grid('True',linewidth=2)
   #CS = ax.contour(Kx_mesh, Ky_mesh, grid_FS_k_c, [0],colors=['black'],linewidths = [5])
   #plt.plot(x, y, c = 'green', linestyle = 'dashdot',linewidth = 5)
   #CS_1 = ax.contour(Kx_mesh, Ky_mesh, radial_FS_del, [0.0], colors = ['green'], linestyles = ['dashdot'],linewidths = [5])
   #CS_2 = ax.contour(Kx_mesh, Ky_mesh, radial_g_k_del, [0.0], colors = ['orange'], linestyles = ['dashdot'],linewidths = [5])
   #CS_3 = ax.contour(Kx_mesh_ni, Ky_mesh_ni, Ek_NI, [mu_ni_actual], colors = ['white'], linestyles = ['dashdot'],linewidths = [5])
   plt.xticks([0,np.pi/4,np.pi/2,3*np.pi/4,np.pi],["0",r"$\pi/4$",r"$\pi/2$",r"$3\pi/4$",r"$\pi$"],fontsize = 80)
   plt.yticks([0,np.pi/4,np.pi/2,3*np.pi/4,np.pi],["0",r"$\pi/4$",r"$\pi/2$",r"$3\pi/4$",r"$\pi$"],fontsize = 80)
   divider = make_axes_locatable(ax)
   cax = divider.append_axes("right", size="5%", pad=0.05)
   cbar = plt.colorbar(im, cax= cax)
   cbar.ax.tick_params(labelsize = 60)
   plt.tight_layout()
   plt.savefig("%s/Gaussian_sigma_%s_spectral_functions_fermi_surface_N_%s_U_%s_mu_%s_dtau_%s_L_%s_omega_max_%s_omega_min_%s.png"%(Graph_dir_gaus_fs_cont,sigma,N,u,mu,dtau,trot_slice,omega_max,omega_min))
   plt.close('all')


   

In [102]:
def real_part_G_plot(Text_dir_gf_k_avg_ac_main,Graph_dir_main,n_den,freq_no,sigma,N,u,mu,trot_slice,dtau,omega_max,omega_min):

   beta = float(dtau)*float(trot_slice)
   T = 1/beta
   
   mu_ni_actual = fsolve(number_density_func,[0.0],args = (n_den,beta))
   nd_ni_actual = number_density_calc(mu_ni_actual[0],beta)

   print(nd_ni_actual,n_den,"Non_interacting, interacting density")
   bz,k_pairs = k_point_grid_upper_half_bz(N)
   k_label_0 = bz[:,0]
   Kx_grid_0 = bz[:,1]
   Ky_grid_0 = bz[:,2]

   k_label_1 = []
   Kx_grid_1 = []
   Ky_grid_1 = []


   for ii in range(len(k_label_0)):

       if(Kx_grid_0[ii] != Ky_grid_0[ii]):
          kx = Ky_grid_0[ii]
          ky = Kx_grid_0[ii]
          Kx_grid_1.append(kx)
          Ky_grid_1.append(ky)
          k_label_1.append(k_label_0[ii])


   k_label = np.append(k_label_0,np.array(k_label_1))
   Kx_grid_2 = np.append(Kx_grid_0,np.array(Kx_grid_1))
   Ky_grid_2 = np.append(Ky_grid_0,np.array(Ky_grid_1))

   Kx_grid = (2*np.pi/int(N))*Kx_grid_2
   Ky_grid = (2*np.pi/int(N))*Ky_grid_2

   K_lab_Pi,Kx_pi,Ky_pi = k_point_pi_pi_locator(k_label,Kx_grid_2,Ky_grid_2)
        
   Omega_gaus = np.zeros((len(K_lab_Pi),freq_no))
   A_k_gaus = np.zeros((len(K_lab_Pi),freq_no))
   Real_part_G = np.zeros((len(K_lab_Pi),freq_no))
    
   #======================Read the maxent data files one by one==========================================
   for kk in range(len(K_lab_Pi)):

       k_point = str(int(K_lab_Pi[kk]))

       filename_a_k_avg_ac_gaus = '%s/k_point_%s/Spectral_function_momentum_space_avg_N_%s_U_%s_mu_%s_dtau_%s_L_%s_k_label_%s_gaussian_sigma_%s_nfreq_%s_omega_max_%s_omega_min_%s.dat' %(Text_dir_gf_k_avg_ac_main,k_point,N,u,mu,dtau,trot_slice,k_point,sigma,str(freq_no),omega_max,omega_min)
       omega_gaus,a_k_gaus = np.loadtxt(filename_a_k_avg_ac_gaus,unpack = 'True',usecols = [0,1])

       Omega_gaus[kk,:] = np.copy(omega_gaus)
       A_k_gaus[kk,:] = np.copy(a_k_gaus)
       zero_ind_gaus,zero_val_gaus = find_nearest(omega_gaus,0.00000001)

   for kk in range(len(K_lab_Pi)): 
       for w in range(freq_no):
           #Integrand = np.zeros(freq_no)
           #for w2 in range(freq_no):
           #    Integrand[w2] = A_k_gaus[kk,w2]/(Omega_gaus[kk,w]-Omega_gaus[kk,w2]+0.000001*1j)
           Real_part_G[kk][w] = np.real(integrate.trapz(np.divide(A_k_gaus[kk,:],np.subtract((Omega_gaus[kk,w]+0.00000001*1j)*np.ones(freq_no),Omega_gaus[kk,:])),Omega_gaus[kk,:]))


    
   Graph_dir_gaus_fs = "%s/Gaussian_sigma_%s_real_part_retarded_green_function"%(Graph_dir_main,sigma)
   if not os.path.exists(Graph_dir_gaus_fs):
      os.makedirs(Graph_dir_gaus_fs)

   plt.figure(figsize = (25,20))
   plt.title(r"$Re G(k,\omega), N = %sx%s, U = %s, \mu=%s, n =%s, T=%s$" %(N,N,u,mu,str(round(n_den,4)),str(round(T,3))),fontsize = 40)
   plt.xticks(fontsize = 60)
   plt.yticks(fontsize = 40) 
   plt.xlabel(r"$\omega$",fontsize = 80)
   plt.ylabel(r"$ReG(k,\omega)$",fontsize = 80)
   for kk in range(len(K_lab_Pi)): 
       kx_cord = str(round(2*np.pi/int(N),2)*Kx_pi[kk])
       ky_cord = str(round(2*np.pi/int(N),2)*Ky_pi[kk])
       plt.plot(Omega_gaus[kk,:],Real_part_G[kk,:],marker = "o",markersize = 5, label=r"$k_x=%s, k_y=%s$"%(kx_cord,ky_cord))
   plt.grid('True')
   plt.xlim(-10,10)
   plt.legend(loc = 'best',fontsize=40) 
   plt.savefig("%s/Gaussian_sigma_%s_real_part_green's_function_N_%s_U_%s_mu_%s_dtau_%s_L_%s_omega_max_%s_omega_min_%s.png"%(Graph_dir_gaus_fs,sigma,N,u,mu,dtau,trot_slice,omega_max,omega_min))
   plt.close('all')


    

In [103]:
def spectral_func_plot(Text_dir_gf_k_avg_ac_main,Graph_dir_main,n_den,freq_no,sigma,N,u,mu,trot_slice,dtau,omega_max,omega_min):

   beta = float(dtau)*float(trot_slice)
   T = 1/beta
   
   mu_ni_actual = fsolve(number_density_func,[0.0],args = (n_den,beta))
   nd_ni_actual = number_density_calc(mu_ni_actual[0],beta)

   print(nd_ni_actual,n_den,"Non_interacting, interacting density")
   bz,k_pairs = k_point_grid_upper_half_bz(N)
   k_label_0 = bz[:,0]
   Kx_grid_0 = bz[:,1]
   Ky_grid_0 = bz[:,2]

   k_label_1 = []
   Kx_grid_1 = []
   Ky_grid_1 = []


   for ii in range(len(k_label_0)):

       if(Kx_grid_0[ii] != Ky_grid_0[ii]):
          kx = Ky_grid_0[ii]
          ky = Kx_grid_0[ii]
          Kx_grid_1.append(kx)
          Ky_grid_1.append(ky)
          k_label_1.append(k_label_0[ii])


   k_label = np.append(k_label_0,np.array(k_label_1))
   Kx_grid_2 = np.append(Kx_grid_0,np.array(Kx_grid_1))
   Ky_grid_2 = np.append(Ky_grid_0,np.array(Ky_grid_1))

   Kx_grid = (2*np.pi/int(N))*Kx_grid_2
   Ky_grid = (2*np.pi/int(N))*Ky_grid_2

   K_lab_Pi,Kx_pi,Ky_pi = k_point_pi_pi_locator(k_label,Kx_grid_2,Ky_grid_2)
        
   Omega_gaus = np.zeros((len(K_lab_Pi),freq_no))
   A_k_gaus = np.zeros((len(K_lab_Pi),freq_no))
   Real_part_G = np.zeros((len(K_lab_Pi),freq_no))
    
   #======================Read the maxent data files one by one==========================================
   for kk in range(len(K_lab_Pi)):

       k_point = str(int(K_lab_Pi[kk]))

       filename_a_k_avg_ac_gaus = '%s/k_point_%s/Spectral_function_momentum_space_avg_N_%s_U_%s_mu_%s_dtau_%s_L_%s_k_label_%s_gaussian_sigma_%s_nfreq_%s_omega_max_%s_omega_min_%s.dat' %(Text_dir_gf_k_avg_ac_main,k_point,N,u,mu,dtau,trot_slice,k_point,sigma,str(freq_no),omega_max,omega_min)
       omega_gaus,a_k_gaus = np.loadtxt(filename_a_k_avg_ac_gaus,unpack = 'True',usecols = [0,1])

       Omega_gaus[kk,:] = np.copy(omega_gaus)
       A_k_gaus[kk,:] = np.copy(a_k_gaus)

   beta = float(dtau)*float(trot_slice) 
   Kernel = np.zeros(freq_no)
   for w in range(freq_no):
       Kernel[w] = 0.5/np.cosh(0.5*beta*Omega_gaus[0,w])
       
   Graph_dir_gaus_fs = "%s/Gaussian_sigma_%s_real_part_retarded_green_function"%(Graph_dir_main,sigma)
   if not os.path.exists(Graph_dir_gaus_fs):
      os.makedirs(Graph_dir_gaus_fs)

   plt.figure(figsize = (25,20))
   plt.title(r"$A(k,\omega), N = %sx%s, U = %s, \mu=%s, n =%s, T=%s$" %(N,N,u,mu,str(round(n_den,4)),str(round(T,3))),fontsize = 40)
   plt.xticks(fontsize = 60)
   plt.yticks(fontsize = 40) 
   plt.xlabel(r"$\omega$",fontsize = 80)
   plt.ylabel(r"$A(k,\omega)$",fontsize = 80)
   for kk in range(len(K_lab_Pi)): 
       kx_cord = str(round(2*np.pi/int(N),2)*Kx_pi[kk])
       ky_cord = str(round(2*np.pi/int(N),2)*Ky_pi[kk])
       plt.plot(Omega_gaus[kk,:],A_k_gaus[kk,:],marker = "o",markersize = 5, label=r"$k_x=%s, k_y=%s$"%(kx_cord,ky_cord))
   plt.plot(Omega_gaus[0,:],Kernel,c='black',linestyle = "dashdot",linewidth=3,label=r"$y=\frac{sech (\beta \omega/2)}{2}$")
   plt.grid('True')
   plt.xlim(-10,10)
   plt.legend(loc = 'best',fontsize=40) 
   plt.savefig("%s/Gaussian_sigma_%s_spectral_function_N_%s_U_%s_mu_%s_dtau_%s_L_%s_omega_max_%s_omega_min_%s.png"%(Graph_dir_gaus_fs,sigma,N,u,mu,dtau,trot_slice,omega_max,omega_min))
   plt.close('all')


    

In [104]:
def spectral_func_plot_specific_k(Text_dir_gf_k_avg_ac_main,Graph_dir_main,n_den,freq_no,sigma,N,u,mu,trot_slice,dtau,omega_max,omega_min,
                                 kx_cord_i,ky_cord_i,kx_cord_ni,ky_cord_ni):

   beta = float(dtau)*float(trot_slice)
   T = 1/beta
   
   mu_ni_actual = fsolve(number_density_func,[0.0],args = (n_den,beta))
   nd_ni_actual = number_density_calc(mu_ni_actual[0],beta)

   print(nd_ni_actual,n_den,"Non_interacting, interacting density")
   bz,k_pairs = k_point_grid_upper_half_bz(N)
   k_label = bz[:,0]
   Kx_grid = bz[:,1]
   Ky_grid = bz[:,2]



   k_lab_i = []
   k_lab_ni = [] 
    
   for i in range(len(kx_cord_i)):
       for j in range(len(Kx_grid)):
           if round(kx_cord_i[i],1) == round(Kx_grid[j],1) and round(ky_cord_i[i],1) == round(Ky_grid[j],1):
               k_lab_i.append(k_label[j])

   for i in range(len(kx_cord_ni)):
       for j in range(len(Kx_grid)):
           if round(kx_cord_ni[i],1) == round(Kx_grid[j],1) and round(ky_cord_ni[i],1) == round(Ky_grid[j],1):
               k_lab_ni.append(k_label[j])

   print("interacting k label",k_lab_i)
   print("non interacting k label",k_lab_ni) 
    
   Omega_gaus_i = np.zeros((len(k_lab_i),freq_no))
   A_k_gaus_i = np.zeros((len(k_lab_i),freq_no))

   Omega_gaus_ni = np.zeros((len(k_lab_ni),freq_no))
   A_k_gaus_ni = np.zeros((len(k_lab_ni),freq_no))   
   
   #======================Read the maxent data files one by one for interacting contours==========================================
   for k in range(len(k_lab_i)):

       k_point = str(int(k_lab_i[k]))

       filename_a_k_avg_ac_gaus = '%s/k_point_%s/Spectral_function_momentum_space_avg_N_%s_U_%s_mu_%s_dtau_%s_L_%s_k_label_%s_gaussian_sigma_%s_nfreq_%s_omega_max_%s_omega_min_%s.dat' %(Text_dir_gf_k_avg_ac_main,k_point,N,u,mu,dtau,trot_slice,k_point,sigma,str(freq_no),omega_max,omega_min)
       omega_gaus,a_k_gaus = np.loadtxt(filename_a_k_avg_ac_gaus,unpack = 'True',usecols = [0,1])

       Omega_gaus_i[k,:] = np.copy(omega_gaus)
       A_k_gaus_i[k,:] = np.copy(a_k_gaus)

   #======================Read the maxent data files one by one for non interacting contours==========================================
   for kk in range(len(k_lab_i)):

       k_point = str(int(k_lab_ni[kk]))

       filename_a_k_avg_ac_gaus = '%s/k_point_%s/Spectral_function_momentum_space_avg_N_%s_U_%s_mu_%s_dtau_%s_L_%s_k_label_%s_gaussian_sigma_%s_nfreq_%s_omega_max_%s_omega_min_%s.dat' %(Text_dir_gf_k_avg_ac_main,k_point,N,u,mu,dtau,trot_slice,k_point,sigma,str(freq_no),omega_max,omega_min)
       omega_gaus,a_k_gaus = np.loadtxt(filename_a_k_avg_ac_gaus,unpack = 'True',usecols = [0,1])

       Omega_gaus_ni[kk,:] = np.copy(omega_gaus)
       A_k_gaus_ni[kk,:] = np.copy(a_k_gaus)


    
   Graph_dir_gaus_fs = "%s/Gaussian_sigma_%s_specific_k_point_spectral_function"%(Graph_dir_main,sigma)
   if not os.path.exists(Graph_dir_gaus_fs):
      os.makedirs(Graph_dir_gaus_fs)

   plt.figure(figsize = (25,20))
   plt.title(r"$A(k,\omega), N = %sx%s, U = %s, \mu=%s, n =%s, T=%s$" %(N,N,u,mu,str(round(n_den,4)),str(round(T,3))),fontsize = 40)
   plt.xticks(fontsize = 60)
   plt.yticks(fontsize = 40) 
   plt.xlabel(r"$\omega$",fontsize = 80)
   plt.ylabel(r"$A(k,\omega)$",fontsize = 80)
   for k in range(len(k_lab_i)): 
       kx_cord = str(round(2*np.pi/int(N),2)*kx_cord_i[k])
       ky_cord = str(round(2*np.pi/int(N),2)*ky_cord_i[k])
       plt.plot(Omega_gaus_i[k,:],A_k_gaus_i[k,:],marker = "o",markersize = 10, label=r"$k_x=%s, k_y=%s$, int FS"%(kx_cord,ky_cord))
   
   for kk in range(len(k_lab_ni)): 
       kx_cord = str(round(2*np.pi/int(N),2)*kx_cord_ni[kk])
       ky_cord = str(round(2*np.pi/int(N),2)*ky_cord_ni[kk])
       plt.plot(Omega_gaus_ni[kk,:],A_k_gaus_ni[kk,:],marker = "^",markersize = 10, label=r"$k_x=%s, k_y=%s$, NI FS"%(kx_cord,ky_cord))
   plt.axvline(x=0,color='black',linestyle='dashed',linewidth=3)
   plt.grid('True')
   plt.xlim(-15,15)
   plt.legend(loc = 'best',fontsize=40) 
   plt.savefig("%s/Gaussian_sigma_%s_spectral_function_specific_points_N_%s_U_%s_mu_%s_dtau_%s_L_%s_omega_max_%s_omega_min_%s.png"%(Graph_dir_gaus_fs,sigma,N,u,mu,dtau,trot_slice,omega_max,omega_min))
   plt.close('all')


    

In [109]:
def main():

    N = 10
    U = ["4.00"] #,"5.00","5.50","6.00","6.50","8.00"]
    Mu = ["0.00"] #,"0.20","0.40","0.60","0.80","1.00","1.20","1.40","1.60","1.80","2.00"] #,"2.20","2.40","2.60","2.80","3.00","3.20","3.40","3.60","3.80","4.00","4.20","4.40","4.60","5.00","5.20","5.40","5.60","5.80"] #,"6.00","6.20","6.40","6.60"]
    Trot = ["30"]
    FS_area = np.zeros((len(Mu),len(Trot)))
    FS_area_NI = np.zeros((len(Mu),len(Trot)))
    Nden = np.zeros((len(Mu),len(Trot)))
    
    Dtau = "0.05"
    Omega_max = "25.0"
    Omega_min = "-25.0"
    Freq_no = 2500
    
    XGrid_size = 20
    YGrid_size = 20
    Sigma = ["2.0"]
    
    KX_I = [[1],[0],[1],[2],[1],[2],[1]]
    KY_I = [[1],[2],[3],[3],[4],[4],[6]]
    KX_NI = [[3],[3],[1],[1],[1],[2],[2]]
    KY_NI = [[3],[3],[6],[6],[6],[5],[5]]


    for u in range(len(U)):
        for sig in range(len(Sigma)):
        
            Graph_dir_main = "/Users/roy.369/Documents/Cold_atom_correlators/FHM_Legacy_data/FHM_Roy_data/Graphs/Graphs_N_%s_maxent/Graphs_N_%s_U_%s_dtau_%s_maxent_spectral_function_plots/Graphs_N_%s_U_%s_dtau_%s_maxent_plots_fermi_surface/Spectral_functions_fermi_surface_gaussian_sigma_%s_default_model_omega_max_%s_omega_min_%s"%(N,N,U[u],Dtau,N,U[u],Dtau,Sigma[sig],Omega_max,Omega_min)
        
            Graph_dir_main_kf = "/Users/roy.369/Documents/Cold_atom_correlators/FHM_Legacy_data/FHM_Roy_data/Graphs/Graphs_N_%s_maxent/Graphs_N_%s_U_%s_dtau_%s_maxent_spectral_function_plots/Graphs_N_%s_U_%s_dtau_%s_maxent_plots_fermi_surface/Spectral_functions_fermi_momenta_gaussian_sigma_%s_default_model_omega_max_%s_omega_min_%s"%(N,N,U[u],Dtau,N,U[u],Dtau,Sigma[sig],Omega_max,Omega_min)
            if not os.path.exists(Graph_dir_main_kf):
               os.makedirs(Graph_dir_main_kf)
           
            for k in range(len(Trot)):
        
                Graph_dir_ac_fs = "%s/dtau_%s_L_%s"%(Graph_dir_main,Dtau,Trot[k])
                if not os.path.exists(Graph_dir_ac_fs):
                    os.makedirs(Graph_dir_ac_fs)
            
                Graph_dir_ac_kf = "%s/dtau_%s_L_%s"%(Graph_dir_main_kf,Dtau,Trot[k])
                if not os.path.exists(Graph_dir_ac_kf):
                   os.makedirs(Graph_dir_ac_kf)

    
                for i in range(len(Mu)):
        
                    Text_dir_gf_k_avg_ac_main = "/Users/roy.369/Documents/Cold_atom_correlators/FHM_Legacy_data/FHM_Roy_data/Text_files/Text_files_N_%s_analytic_continuation/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Spectral_functions_gaussian_sigma_%s_nfreq_%s"%(N,N,U[u],Dtau,Mu[i],Dtau,Trot[k],Sigma[sig],str(Freq_no))
                
                    Text_dir_eqm = "/Users/roy.369/Documents/Cold_atom_correlators/FHM_Legacy_data/FHM_Roy_data/Text_files/Text_files_N_%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Thermodynamic_measurements"%(N,N,U[u],Dtau,Mu[i],Dtau,Trot[k])
                    filename_eqm_avg = '%s/Thermodynamic_measurements_normal_averaged_dictionary_N_%s_U_%s_mu_%s_dtau_%s_L_%s.pkl' %(Text_dir_eqm,N,U[u],Mu[i],Dtau,Trot[k])
                    with open(filename_eqm_avg, 'rb') as infile:
                         sys_measure_avg = pickle.load(infile)

                    filename_eqm_std = '%s/Thermodynamic_measurements_normal_standard_deviation_dictionary_N_%s_U_%s_mu_%s_dtau_%s_L_%s.pkl' %(Text_dir_eqm,N,U[u],Mu[i],Dtau,Trot[k])
                    with open(filename_eqm_std, 'rb') as infile:
                         sys_measure_std = pickle.load(infile)

                    Density = sys_measure_avg['Density averaged']
                    Density_dev = sys_measure_std['Density standard deviation']
                    Avg_sign = sys_measure_avg['Total sign averaged']

                    luttinger_surface_spectral_functions_full_bz_plot_v2(Text_dir_gf_k_avg_ac_main,Graph_dir_ac_fs,Density,Freq_no,Sigma[sig],N,U[u],Mu[i],Trot[k],Dtau,Omega_max,Omega_min,XGrid_size,YGrid_size)
                




In [110]:
main()   

[1.5628884e-14] 1.0000000000000084 1.0 Non_interacting, interacting density
[0. 1. 2. 3. 4. 5.]


In [32]:
x=[1]
for i in range(len(x)):
    print(i)

0
